In [4]:
import json, torch

device = "cpu"
dtype  = torch.float64

with open("model.json", "r") as f:
    model = json.load(f)

# and whatever you used to build state_index, n_states, etc.


In [5]:
import torch

theta_teacher_loaded = torch.load("theta_teacher.pt")  # or "/mnt/data/theta_teacher.pt"


In [6]:
theta_teacher_loaded = {
    name: val.to(device=device, dtype=dtype)
    for name, val in theta_teacher_loaded.items()
}


In [11]:
# Base "true" parameters used to generate x_data (teacher)
theta_true = theta_teacher_loaded   # <---- use the loaded dict, not theta_teacher

# Create learnable copy with ~10% multiplicative noise
theta_learn = torch.nn.ParameterDict()

noise_level = 0.10  # 10% noise

for name, val in theta_true.items():
    # Only make learnable if it's in the "learnable" group from model.json
    if name in model["parameters_fit_groups"]["learnable"]:
        v0 = val.clone().detach().to(device=device, dtype=dtype)
        noise = noise_level * torch.randn_like(v0)
        init_val = v0 * (1.0 + noise)
        theta_learn[name] = torch.nn.Parameter(init_val)
    else:
        # keep fixed params as plain tensors
        theta_learn[name] = val.clone().detach().to(device=device, dtype=dtype)

theta_learn


ParameterDict(
    (rT_max): Parameter containing: [torch.DoubleTensor of size ]
    (k_ET): Parameter containing: [torch.DoubleTensor of size ]
    (K_ET): Parameter containing: [torch.DoubleTensor of size ]
    (k_Tam_ERPR): Parameter containing: [torch.DoubleTensor of size ]
    (K_Tam): Parameter containing: [torch.DoubleTensor of size ]
    (rE_max): Parameter containing: [torch.DoubleTensor of size ]
    (rH_max): Parameter containing: [torch.DoubleTensor of size ]
    (delta_E): Parameter containing: [torch.DoubleTensor of size ]
    (delta_H): Parameter containing: [torch.DoubleTensor of size ]
    (k_T_to_E): Parameter containing: [torch.DoubleTensor of size ]
    (K_Tstim_E): Parameter containing: [torch.DoubleTensor of size ]
    (k_T_to_H): Parameter containing: [torch.DoubleTensor of size ]
    (K_Tstim_H): Parameter containing: [torch.DoubleTensor of size ]
    (k_H_to_E): Parameter containing: [torch.DoubleTensor of size ]
    (K_Hboost_E): Parameter containing: [torch.D

In [13]:
fit_state_names = ["T", "ctDNA"]

state_mask = torch.ones(1, n_states, dtype=dtype, device=device)
for s in fit_state_names:
    idx = state_index[s]
    state_mask[0, idx] = 1.0

state_mask


NameError: name 'n_states' is not defined